In [2]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

In [13]:
cleaned_crime = pd.read_csv('Resources/crime_chip_for_model.csv')
cleaned_crime.head()

,Unnamed: 0,ZIP,Ward,Primary_Type,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,state,location,address,chip_latitude,chip_longitude,chipotle,Safety
0,0,60601,27,THEFT,3,NaN,NaN,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
1,1,60601,27,CRIMINAL DAMAGE,4,41.883932,-87.679964,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
2,2,60601,27,THEFT,3,41.896569,-87.636063,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
3,3,60601,27,SEX OFFENSE,3,41.883937,-87.683368,True,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
4,4,60601,27,NARCOTICS,4,41.892856,-87.710137,True,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad


In [14]:
cleaned_crime = cleaned_crime.drop(columns=['Unnamed: 0', 'state', 'location', 'address' ,'chip_latitude', 'chip_longitude'])
cleaned_crime = cleaned_crime.dropna()
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].str.replace('#', '')
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].str.replace(',', '')
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].astype('int')
cleaned_crime['People/Sq.Mile'] = cleaned_crime['People/Sq.Mile'].str.replace(',', '')
cleaned_crime['People/Sq.Mile'] = cleaned_crime['People/Sq.Mile'].astype('float')
cleaned_crime['Population'] = cleaned_crime['Population'].str.replace(',', '')
cleaned_crime['Population'] = cleaned_crime['Population'].astype('int')
sample = cleaned_crime.sample(n=30000)
sample = sample.reset_index()
sample = sample.drop(columns=['index'])
sample.head()

,ZIP,Ward,Primary_Type,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,chipotle,Safety
0,60641,6,WEAPONS VIOLATION,5,41.751168,-87.611898,True,False,73824,18163.14,241,1,Bad
1,60641,6,ASSAULT,7,41.759305,-87.621556,False,False,73824,18163.14,241,1,Bad
2,60661,30,BATTERY,3,41.923976,-87.766294,True,False,4382,15455.28,318,1,Bad
3,60612,16,CRIMINAL DAMAGE,4,41.789309,-87.687573,False,True,37990,10195.72,586,1,Bad
4,60625,5,OFFENSE INVOLVING CHILDREN,7,41.771684,-87.571068,False,False,91351,23681.91,156,0,Bad


In [15]:
primary_type = sample.Primary_Type.value_counts()

In [16]:
# Determine which values to replace if counts are less than ...?
replace_primary = list(primary_type[primary_type < 1000].index)

# Replace in dataframe
for primary in replace_primary:
    sample.Primary_Type = sample.Primary_Type.replace(primary,"Other")
    
# Check to make sure binning was successful
sample.Primary_Type.value_counts()

BATTERY                6217
THEFT                  5892
CRIMINAL DAMAGE        3612
ASSAULT                2629
Other                  1994
DECEPTIVE PRACTICE     1854
OTHER OFFENSE          1829
BURGLARY               1394
MOTOR VEHICLE THEFT    1286
NARCOTICS              1187
WEAPONS VIOLATION      1086
ROBBERY                1020
Name: Primary_Type, dtype: int64

In [17]:
# Generate our categorical variable lists
crime_obj = list(sample.dtypes[sample.dtypes == 'object'].index)

In [18]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(cleaned_crime[crime_obj]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(crime_obj)
encode_df.head()

,Primary_Type_ARSON,Primary_Type_ASSAULT,Primary_Type_BATTERY,Primary_Type_BURGLARY,Primary_Type_CONCEALED CARRY LICENSE VIOLATION,Primary_Type_CRIM SEXUAL ASSAULT,Primary_Type_CRIMINAL DAMAGE,Primary_Type_CRIMINAL SEXUAL ASSAULT,Primary_Type_CRIMINAL TRESPASS,Primary_Type_DECEPTIVE PRACTICE,...,Primary_Type_PROSTITUTION,Primary_Type_PUBLIC INDECENCY,Primary_Type_PUBLIC PEACE VIOLATION,Primary_Type_ROBBERY,Primary_Type_SEX OFFENSE,Primary_Type_STALKING,Primary_Type_THEFT,Primary_Type_WEAPONS VIOLATION,Safety_Bad,Safety_Good
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals
sample = sample.merge(encode_df, left_index=True, right_index=True)
sample = sample.drop(columns=crime_obj,axis=1)
sample.head()

,ZIP,Ward,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,...,Primary_Type_PROSTITUTION,Primary_Type_PUBLIC INDECENCY,Primary_Type_PUBLIC PEACE VIOLATION,Primary_Type_ROBBERY,Primary_Type_SEX OFFENSE,Primary_Type_STALKING,Primary_Type_THEFT,Primary_Type_WEAPONS VIOLATION,Safety_Bad,Safety_Good
0,60641,6,5,41.751168,-87.611898,True,False,73824,18163.14,241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60641,6,7,41.759305,-87.621556,False,False,73824,18163.14,241,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,60661,30,3,41.923976,-87.766294,True,False,4382,15455.28,318,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,60612,16,4,41.789309,-87.687573,False,True,37990,10195.72,586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60625,5,7,41.771684,-87.571068,False,False,91351,23681.91,156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Split our preprocessed data into our features and target arrays
X = sample.drop(columns=['chipotle'],axis=1).values
y = sample['chipotle'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Neural Network

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                1760      
                                                                 
 dense_4 (Dense)             (None, 20)                820       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,601
Trainable params: 2,601
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights_opt_1.{epoch:02d}.hdf5"

In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
704/704 [==============================] - 1s 1ms/step - loss: 0.5188 - accuracy: 0.7298
Epoch 2/100
704/704 [==============================] - 1s 1ms/step - loss: 0.2122 - accuracy: 0.9318
Epoch 3/100
704/704 [==============================] - 1s 1ms/step - loss: 0.0775 - accuracy: 0.9796
Epoch 4/100
704/704 [==============================] - 1s 1ms/step - loss: 0.0419 - accuracy: 0.9895
Epoch 5/100
698/704 [============================>.] - ETA: 0s - loss: 0.0294 - accuracy: 0.9954
Epoch 5: saving model to checkpoints\weights_opt_1.05.hdf5
704/704 [==============================] - 1s 1ms/step - loss: 0.0293 - accuracy: 0.9955
Epoch 6/100
704/704 [==============================] - 1s 1ms/step - loss: 0.0202 - accuracy: 0.9966
Epoch 7/100
704/704 [==============================] - 1s 1ms/step - loss: 0.0175 - accuracy: 0.9964
Epoch 8/100
704/704 [==============================] - 1s 1ms/step - loss: 0.0146 - accuracy: 0.9970
Epoch 9/100
704/704 [===========================

704/704 [==============================] - 1s 1ms/step - loss: 7.2703e-04 - accuracy: 0.9997
Epoch 63/100
704/704 [==============================] - 1s 1ms/step - loss: 3.2433e-04 - accuracy: 0.9999
Epoch 64/100
704/704 [==============================] - 1s 1ms/step - loss: 7.5674e-04 - accuracy: 0.9998
Epoch 65/100
689/704 [============================>.] - ETA: 0s - loss: 3.6682e-04 - accuracy: 0.9999
Epoch 65: saving model to checkpoints\weights_opt_1.65.hdf5
704/704 [==============================] - 1s 1ms/step - loss: 3.6025e-04 - accuracy: 0.9999
Epoch 66/100
704/704 [==============================] - 1s 1ms/step - loss: 5.8724e-05 - accuracy: 1.0000
Epoch 67/100
704/704 [==============================] - 1s 2ms/step - loss: 1.4241e-04 - accuracy: 1.0000
Epoch 68/100
704/704 [==============================] - 1s 2ms/step - loss: 5.7313e-04 - accuracy: 0.9998
Epoch 69/100
704/704 [==============================] - 1s 2ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 70/100
687/704

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

235/235 - 0s - loss: 0.0078 - accuracy: 0.9989 - 354ms/epoch - 2ms/step
Loss: 0.0077880145981907845, Accuracy: 0.9989333152770996
